### 1. Don't overfit a sigle batch
### 2. Forgot to set model.eval() or model.train()
### 3. Forgot to set optimizer.zero_grad()
### 4. Using softmax in model with CrossEntropyLoss()
### 5. Don't set bais=False when using batchnorm()
### 6. Using view as permute
### 7. Using bad data augmentation
### 8. Not Shuffling Data
### 9. Not Normalizing data
### 10. Not Clipping gradients (when using RNNs, GRUs, or LSTMs)

In [1]:
# Import
import torch
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F # relu, tanh
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms

In [2]:
# 创建随机数种子
seed=0
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False 
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# Create Fully Connected Network
class NN(nn.Module):
    def __init__(self, input_size, num_classes): 
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x 

In [ ]:
# Create Fully Connected Network
class NN(nn.Module):
    def __init__(self, input_size, num_classes): 
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        self.dropout = nn.Dropout(0.5)
        # 14_How To Save Time
        # 4.Don't use softmax with crossentropy
        # 在计算交叉熵损失时，已经进行了一次softmax操作，再计算的负样本
        # 如果在网络最后的输出层也是softmax，就相当于进行了两次softmax操作
        # 可能会带来梯度消失问题
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.softmax(self.fc2(x))
        return x 

In [3]:
# A cnn network
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x

In [3]:
# A cnn network with batchnorm
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
            # 14_How To Save Time
            # 5.Set bias=False, when using BatcnNorm
            # 使用batchnorm时，bias无效，具体见batchnorm公式
            bias=False,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)
        self.bn1 = nn.BatchNorm2d(8)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x

In [ ]:
model = NN(784, 10)
x = torch.randn(64, 784)
print(model(x).shape)

In [4]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
# Hyperparameters
input_size = 784
num_class = 10
learning_rate = 0.001
batch_size = 256 
num_epochs = 3

In [6]:
# 14_How To Save Time
# 9. Not normalizing data
# 在数据进行训练前，应该进行标准化处理
# 所有图像的像素值除以255，使得数据在0-1之间
# 进而得到数据集的所有图像的像素的均值和方差
# 如果是RGB图像，需要3个通道的均值和方差
# 对于灰度图，如MNIST数据，只需要单个均值和方差
my_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.1307,), std=(0.3081,)),
    ]
)

In [7]:
# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True,
                               transform=transforms.ToTensor(),
                               download=True) 
# 14_How to Save Time
# 8. Not Shuffling Data
# 需要注意的是，在使用时序相关的数据时不会使用shuffle  
# 但在大多是情况下使用shuffle都是应该的
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                          shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False,
                               transform=transforms.ToTensor(),
                               download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size,
                          shuffle=True)

In [8]:
# Load Data
# Using my_transform with normalize
train_dataset = datasets.MNIST(root='dataset/', train=True,
                               transform=my_transform,
                               download=True) 
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                          shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False,
                               transform=my_transform,
                               download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size,
                          shuffle=True)

In [ ]:
train_dataset[0][0].shape, train_dataset[0][0].reshape(-1).shape

In [8]:
# Input type (torch.cuda.FloatTensor) and weight 
# type (torch.FloatTensor) should be the same
model = CNN().to(device)


In [ ]:
# Initialize network
model = NN(input_size=input_size, num_classes=num_class).to(device)

In [9]:
# Loss and optimizer
# 14_How To Save Time
# 4.Don't use softmax with crossentropy
# 在计算交叉熵损失时，已经进行了一次softmax操作，再计算的负样本
# 如果在网络最后的输出层也是softmax，就相当于进行了两次softmax操作
# 可能会带来梯度消失问题
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 14_How To Save Time
# 1. Use overfit a single batch
# 在正式训练前使用一个batch的数据进行测试，调整batchsize的大小

data, targets = next(iter(train_loader))
for epoch in range(num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    data = data.to(device=device)
    targets = targets.to(device=device)

    data = data.reshape(data.shape[0], -1)

    scores = model(data)
    loss = criterion(scores, targets)
    print(loss)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    
check_accuracy(test_loader, model)


In [ ]:
# Train Network
# For NN datashape --> 784
from tqdm import tqdm
for epoch in range(num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]:", end='')
    for data, targets in tqdm(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        # 14_How To Save Time
        # Don't forget optimizer.zero_grad()
        # 在不使用zero_grad()时，优化器用的是过去所有batch梯度的累计
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()





In [ ]:
# Train Network
from tqdm import tqdm
for epoch in range(num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]:", end='')
    for data, targets in tqdm(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()
        # 14_How To Save Time
        # 10. Not clipping gradient (when using RNNs, GRUs, LSTMs)
        # 具体原因晓不得，反正说加了这一行，效果会很好
        torch.nn.utils.clip_grad_norm(model.parameters(), max_norm=1)

        # gradient descent or adam step
        optimizer.step()





In [10]:
# Train Network
# For CNN datashape --> 1 x 28 x 28
from tqdm import tqdm
for epoch in range(num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]:", end='')
    for data, targets in tqdm(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        # 14_How To Save Time
        # Don't forget optimizer.zero_grad()
        # 在不使用zero_grad()时，优化器用的是过去所有batch梯度的累计
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()





Epoch [1/3]:

100%|██████████| 235/235 [00:07<00:00, 32.00it/s]


Epoch [2/3]:

100%|██████████| 235/235 [00:04<00:00, 49.15it/s]


Epoch [3/3]:

100%|██████████| 235/235 [00:04<00:00, 48.82it/s]


In [11]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # model.eval()

    with torch.no_grad():
        if loader.dataset.train:
            print("Checking accuracy on training data")
        else:
            print("Checking accuracy on test data")
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            # x = x.reshape(x.shape[0], -1)

            scores = model(x) # shape: 64x10
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
        
        acc = round(float(num_correct)/float(num_samples)*100, 2)
        print(f'Got {num_correct} / {num_samples} with accuracy \
              {acc}')
    
    # model.train()
    # return acc
            

In [12]:
# check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on test data
Got 9789 / 10000 with accuracy               97.89


In [ ]:
# 14_How To Save Time
# 2. Forgot to set training or eval
# model.training() 模式下会使用dropout,并只对一个batch做normalization
# model.eval() 模式下不会使用dropout，并对整体测试集数据做normalization
model.train()
check_accuracy(test_loader, model)
model.eval()
check_accuracy(test_loader, model)


In [25]:
# 14_How To Save Time
# 6. Using view as permute
# view()是按照元素顺序直接重新排列
# permute()是置换原矩阵的指定维度
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(x)

print(x.view(3, 2))
print(x.permute(1, 0)) # This is transpose, in specify dimention.
print(x.T) # Transpose of a matrix

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [ ]:
# 14_How To Save Time
# 7. Using bad data augmentation
# 在使用数据增强时应考虑训练数据的真实分布情况
# 明白数据增强真的达到了你想要的效果
# 比如对于MNIST数据集，手写数字识别
# 如果做了图像翻转，9的图像变成了6，但标签还是9
# 这样会给模型带去误解
# 数据增强是好的，但并不是所有数据增强都是好的！！！